                                                 capstone project
                                       
                                       Open Asian Food Restaurant in Toronto

In [102]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install BeautifulSoup4
!pip install requests
!pip install lxml
!pip install html5lib
!pip install geopy

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans  # import k-means from clustering stage


import folium # map rendering library

from bs4 import BeautifulSoup # library to parse HTML and XML documents

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Scrape Data from Wikipedia:

In [103]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

Create Pandas Dataframe:


In [104]:
postalCodeList = []
boroughList = []
neighborhoodList = []

soup.find('table').find_all('tr') # find the table

soup.find('table').find_all('tr') # find all the rows of the table


# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
        
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove Boroughs that are 'Not assigned' :

In [105]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


combination group neighborhoods in the same borough :

In [106]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


for Neighborhood Not assigned, make the value the same as Borough:

In [107]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head(6)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village


print the number of rows of the cleaned dataframe

In [108]:
print(toronto_df.shape)

(180, 3)


In [109]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [110]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


create map of Toronto using latitude and longitude values:

In [111]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

filter borough names that contain the word Toronto:


In [112]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

 create a new DataFrame with only boroughs that contain the word Toronto:


In [113]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


create map of Toronto using latitude and longitude values:


In [114]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

define Foursquare Credentials and Version:


In [115]:
CLIENT_ID = 'Z02UNGWYWESAQRHEGCX0V00UCJ5S5WRTXWLQ0SQ3AAOJMP1U' # your Foursquare ID
CLIENT_SECRET = 'GNVVRICKW4FVLAYQELGWIE1CAKIVZ13MXNHOCT45X1OCNYT1'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z02UNGWYWESAQRHEGCX0V00UCJ5S5WRTXWLQ0SQ3AAOJMP1U
CLIENT_SECRET:GNVVRICKW4FVLAYQELGWIE1CAKIVZ13MXNHOCT45X1OCNYT1


get the top 200 venues that are within a radius of 2000 meters:

In [ ]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

convert the venues list into a new DataFrame:


In [ ]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

In [ ]:
# check how many venues were returned for each PostalCode
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

]find out how many unique categories can be curated from all the returned venues:


In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
venues_df['VenueCategory'].unique()[:50]


Analyze Each Area:


In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [ ]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).sum().reset_index()

print(toronto_grouped.shape)
toronto_grouped

create the new dataframe and display the top 10 venues for each PostalCode:


In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

In [ ]:
#Calculate the total number of restaurants in each region
toronto_grouped.columns.values.tolist()
total_restaurants =toronto_grouped['American Restaurant']+toronto_grouped['Asian Restaurant']+toronto_grouped['BBQ Joint']+\
toronto_grouped['Bagel Shop']+toronto_grouped['Brazilian Restaurant']+toronto_grouped['Breakfast Spot']+toronto_grouped['Burger Joint']+\
toronto_grouped['Burrito Place']+toronto_grouped['Cantonese Restaurant']+toronto_grouped['Caribbean Restaurant']+\
toronto_grouped['Chinese Restaurant']+toronto_grouped['Comfort Food Restaurant']+toronto_grouped['Cuban Restaurant']+toronto_grouped['Deli / Bodega']+\
toronto_grouped['Diner']+toronto_grouped['Doner Restaurant']+toronto_grouped['Dumpling Restaurant']+\
toronto_grouped['Eastern European Restaurant']+toronto_grouped['Egyptian Restaurant']+toronto_grouped['Ethiopian Restaurant']+\
toronto_grouped['Falafel Restaurant']+toronto_grouped['Fast Food Restaurant']+\
toronto_grouped['Fish & Chips Shop']+toronto_grouped['French Restaurant']+toronto_grouped['Fried Chicken Joint']+toronto_grouped['Greek Restaurant']+toronto_grouped['Hawaiian Restaurant']+\
toronto_grouped['Hungarian Restaurant']+toronto_grouped['Indian Chinese Restaurant']+\
toronto_grouped['Indian Restaurant']+toronto_grouped['Indonesian Restaurant']+toronto_grouped['Italian Restaurant']+\
toronto_grouped['Japanese Restaurant']+toronto_grouped['Jewish Restaurant']+toronto_grouped['Korean Restaurant']+\
toronto_grouped['Latin American Restaurant']+toronto_grouped['Mediterranean Restaurant']+\
toronto_grouped['Mexican Restaurant']+toronto_grouped['Middle Eastern Restaurant']+toronto_grouped['Modern European Restaurant']+\
toronto_grouped['New American Restaurant']+toronto_grouped['Pakistani Restaurant']+\
toronto_grouped['Persian Restaurant']+toronto_grouped['Peruvian Restaurant']+toronto_grouped['Pizza Place']+\
toronto_grouped['Portuguese Restaurant']+toronto_grouped['Ramen Restaurant']+toronto_grouped['Restaurant']+\
toronto_grouped['Salad Place']+toronto_grouped['Sandwich Place']+toronto_grouped['Seafood Restaurant']+\
toronto_grouped['South American Restaurant']+toronto_grouped['Spanish Restaurant']+toronto_grouped['Steakhouse']+toronto_grouped['Sushi Restaurant']+\
toronto_grouped['Taco Place']+toronto_grouped['Tapas Restaurant']+toronto_grouped['Thai Restaurant']+toronto_grouped['Tibetan Restaurant']+\
toronto_grouped['Turkish Restaurant']+toronto_grouped['Vegetarian / Vegan Restaurant']+toronto_grouped['Vietnamese Restaurant']+toronto_grouped['Wings Joint']

In [ ]:
df_restaurants = pd.DataFrame(data={'Neighborhood': toronto_grouped["Neighborhoods"],'Total Restaurants':total_restaurants, 'Asian Restaurants':toronto_grouped['Asian Restaurant']})
df_restaurants.head(10)

In [ ]:
# Create the final data frame that will be analyzed using clustering
df_final = pd.merge(toronto_df_new,df_restaurants,on='Neighborhood')
df_final.head()

In [ ]:
#Drop columns that wont be used for clustering
df_final1=df_final.drop(['PostalCode','Borough','Latitude','Longitude'],axis=1)
df_final1.head()

Cluster Areas:


In [ ]:
# set number of clusters
kclusters = 5

grouped_clustering = df_final1.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [ ]:
venues_merged = df_final.loc[:,'PostalCode':'Longitude']
venues_merged.head()

In [ ]:

# add clustering labels
venues_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
venues_merged = venues_merged.join(df_final1.set_index('Neighborhood'), on='Neighborhood')

print(venues_merged.shape)
venues_merged.head() # check the last columns!

sort the results by Cluster Labels:

In [ ]:
print(venues_merged.shape)
venues_merged.sort_values(["Cluster Labels"], inplace=True)
venues_merged

Finally, visualize the resulting clusters:

create map:


In [ ]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['PostalCode'], venues_merged['Borough'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters:

Cluster 1:

In [ ]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

Cluster 2:

In [ ]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

Cluster 3:

In [ ]:
venues_merged.loc[venues_merged['Cluster Labels'] == 2, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

Cluster 4:

In [ ]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

Cluster 5:

In [ ]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]